<a href="https://colab.research.google.com/github/MeGaNeKoS/Google_Colab/blob/master/GDrive_Direct_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDrive Downloader 

In [ ]:
#@title Mount Google Drive
#@markdown To stream files we need to mount Google Drive.

from google.colab import drive
drive.mount('/content/drive')

## Start Download

In [ ]:
#@title Torrent download
link = "" #@param {type:"string"}
path = "/content/drive/Shared drives/Anime/Torrent" #@param {type:"string"}
Torrent = False #@param {type:"boolean"}
#@markdown It will try to use torrent files first and use the magnet link if file error.
try:
  import libtorrent as lt
except:
  !apt install python3-libtorrent
  import libtorrent as lt
import time
import sys
import datetime
from IPython.display import clear_output
rage = False
clear_output()
begin = time.time()
ses = lt.session()
ses.listen_on(6881, 6891)
def duration(num,msg):
  if int((num)//3600)>0:
    return f"{msg}{int((num)//3600)} hour : {int((num)//60)%60} min : {int((num)%60)} sec"
  elif int((num)//60)>0:
    return f"{msg}{int((num)//60)} min : {int((num)%60)} sec"
  else:
    return f"{msg}{int((num)%60)} sec"

if Torrent:
  from google.colab import files
  uploaded = files.upload()

try:
  info = lt.torrent_info(list(uploaded.keys())[0])
  h = ses.add_torrent({"ti": info, "save_path": path})
  print("starting", h.name())

except:
  #link = input("Paste the magnet link: ") 
  h = lt.add_magnet_uri(ses, link, {"save_path": path})
  if not link:
    print("link incorrect/empty")
    rage = True
  else:
    timeout = time.time() + 60*1
    starting = f'Downloading'
    while (not h.has_metadata()):
      time.sleep(.1)
      if time.time() > timeout:
        rage = True
        starting = f'Failed to download '
        print("Can't get file info")
        break
    info = h.get_torrent_info()
    print(starting, h.name())
  
while not h.is_seed():
    if rage:
      break
    s = h.status()
    state_str = [
        "queued",
        "checking",
        "downloading metadata",
        "downloading",
        "finished",
        "seeding",
        "allocating",
        "checking fastresume",
    ]
    try:
      x = round(((s.total_done/((s.progress * 100)/100)) - s.total_done)/s.download_rate)
      if x > 359999:
        filetime = duration(359999,'')
      else:
        filetime = duration(x,'')
    except:
      filetime = duration(359999,'')
    end = time.time()
    #temp = f"{s.progress * 100} % complete (down: {s.download_rate / 1024} kB/s up: {s.upload_rate / 1024} peers: {s.num_peers}\n{filetime} remaining"
    #print (temp,end='')
    print(
        "\r%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s : %s"
        % (
            s.progress * 100,
            s.download_rate / 1024,
            s.upload_rate / 1024,
            s.num_peers,
            state_str[s.state],
            filetime,
        ), end =' remaining'
      )
    #time.sleep(1)
    #print ("\b"*len(temp),end='')

end = time.time()
temp = duration(end-begin,"Download Time : ")
print('\n'+temp)
#disable due bug
#from google.colab import output
#print(h.name(), "complete")
#output.eval_js('new Audio("https://drive.google.com/uc?export=download&id=1h-KhKaO-FNMsRKKYtiZ-AqNy8tr5mHcX").play()')